In [1]:
import numpy as np
import pandas as pd
from keras.models import Model
from keras.layers import Input, Conv1D, LSTM, Dense, Flatten
from sklearn.preprocessing import RobustScaler

In [2]:
def create_dataset(data, time_steps=5):
    """데이터셋 생성 함수"""
    X, y = [], []
    for i in range(len(data) - time_steps):
        X.append(data[i:(i + time_steps)])
        if data[i + time_steps, 0] > data[i + time_steps - 1, 0]:
            target = 2
        elif data[i + time_steps, 0] < data[i + time_steps - 1, 0]:
            target = 1
        else:
            target = 0
        y.append(target)
    return np.array(X), np.array(y)

In [3]:
def build_model(input_shape):
    """모델 구축 함수 (함수형 API 사용)"""
    inputs = Input(shape=input_shape)
    x = Conv1D(filters=32, kernel_size=5, activation='relu')(inputs)
    x = LSTM(32)(x)
    outputs = Dense(3, activation='softmax')(x)  # 다음 1분 후의 종가 예측

    model = Model(inputs, outputs)
    model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])
    return model

In [4]:
def train_model(model, X, y, epochs=50, batch_size=16):
    """모델 학습 함수"""
    model.fit(X, y, epochs=epochs, batch_size=batch_size, validation_split=0.2)

In [5]:
def make_predictions(model, X):
    """예측 함수"""
    predictions = model.predict(X)
    return predictions

In [6]:
def scale_data(data):
    """데이터 스케일링 함수 (Robust Scaler 사용)"""
    scaler = RobustScaler()
    scaled_data = scaler.fit_transform(data)
    return scaled_data

In [7]:
"""메인 실행"""
# 데이터 준비
data = pd.read_csv('C:\\big18\\final\\with\\000660.csv', encoding='cp949')
data = data[['종가','거래량']]

In [8]:
data

,종가,거래량
0,96800,99206
1,97000,55648
2,96200,37596
3,96300,28194
4,96400,19232
...,...,...
186835,193200,11360
186836,193000,32191
186837,192800,9585
186838,193000,16917


In [9]:
# 데이터 스케일링
scaled_data = scale_data(data)

In [10]:
scaled_data

array([[-0.34899329, 12.15099898],
       [-0.34563758,  6.57236168],
       [-0.3590604 ,  4.26037398],
       ...,
       [ 1.26174497,  0.67289959],
       [ 1.26510067,  1.61193648],
       [ 1.26845638,  1.56621414]])

In [11]:
# 데이터셋 생성
time_steps = 100
X, y = create_dataset(scaled_data, time_steps=time_steps)

In [12]:
# 모델 구축
model = build_model(input_shape=(X.shape[1], X.shape[2]))

In [13]:
# 모델 학습
train_model(model, X, y, epochs=100, batch_size=32)

Epoch 1/100
4669/4669 [==============================] - 45s 9ms/step - loss: 1.0709 - accuracy: 0.4340 - val_loss: 1.0932 - val_accuracy: 0.3838
Epoch 2/100
4669/4669 [==============================] - 41s 9ms/step - loss: 1.0700 - accuracy: 0.4339 - val_loss: 1.1035 - val_accuracy: 0.3679
Epoch 3/100
4405/4669 [===========================>..] - ETA: 2s - loss: 1.0698 - accuracy: 0.4347

KeyboardInterrupt: 

In [ ]:

# 예측
predictions = make_predictions(model, X)
print(predictions)

model.summary()